In [1]:
corpus_fname = '/home/paulkim/workspace/python/Korean_NLP/data/corpus_10days/news/2016-10-24_article_all_normed.txt'
mm_fname = '/home/paulkim/workspace/python/Korean_NLP/data/corpus_10days/models/2016-10-24_article_all_normed_corpus.mtx'
mm_vocab = '/home/paulkim/workspace/python/Korean_NLP/data/corpus_10days/models/2016-10-24_article_all_normed_corpus.vocab'
dictionary_fname = '/home/paulkim/workspace/python/Korean_NLP/data/corpus_10days/models/2016-10-24_article_all_normed_corpus.dictionary'
ldamodel_fname = '/home/paulkim/workspace/python/Korean_NLP/data/corpus_10days/models/2016-10-24_article_all_normed_corpus_lda.pkl'

import pickle
from corpus import Corpus
from sklearn.feature_extraction.text import CountVectorizer
from scipy.io import mmwrite, mmread

PREPROCESSING = True # False
TRAIN_LDA = True # False

In [2]:
if PREPROCESSING:
    corpus = Corpus(corpus_fname, iter_sent=False)
    print('num docs = ', len(corpus))
    
    with open('tmp/extracted_noun_dict.pkl', 'rb') as f:
        noun_dict = pickle.load(f)
        
    def custom_tokenize(doc):
        def parse_noun(token):
            for e in reversed(range(1, len(token) + 1)):
                subword = token[:e]
                if subword in noun_dict:
                    return subword
            return ''
        
        nouns = [parse_noun(token) for token in doc.split()]
        nouns = [word for word in nouns if word]
        return nouns
    
    
    vectorizer = CountVectorizer(tokenizer=custom_tokenize)
    x = vectorizer.fit_transform(corpus)
    mmwrite(mm_fname, x)
    with open(mm_vocab, 'w', encoding='utf-8') as f:
        for word, _ in sorted(vectorizer.vocabulary_.items(), key=lambda x:x[1]):
            f.write('%s\n'%word)
            
    print(x.shape)
    
else:
    x = mmread(mm_fname)
    print(x.shape)

num docs =  26368
(26368, 4760)


## Gensim tutorial

### [Corpus Formats][corpus_format]

Gensim의 LDA가 이용하는 학습데이터의 형식은 list of list of tuple이며, tuple은 (term_index, frequency) 임


    >>> from gensim import corpora
    
    >>> # create a toy corpus of 2 documents, as a plain Python list
    >>> corpus = [[(1, 0.5)], []]  # make one document empty, for the heck of it
    >>>
    >>> corpora.MmCorpus.serialize('/tmp/corpus.mm', corpus)
    
혹은 만약 sparse matrix를 만든 다음, mmwrite를 이용하여 저장하였다면 아래처럼 term frequency matrix 파일을 불러들일수도 있습니다. 


    >>> scipy_sparse_matrix = mmread(mm_fname)    
    >>> corpus = gensim.matutils.Sparse2Corpus(scipy_sparse_matrix)

그런데, gensim은 sparse matrix를 (doc, term) matrix가 아니라 (term, doc) matrix라고 가정합니다. 그래서 sparse matrix를 transpose() 해야 합니다. 

    >>> corpus = gensim.matutils.Sparse2Corpus(scipy_sparse_matrix.transpose())



[corpus_format]: https://radimrehurek.com/gensim/tut1.html#corpus-formats

In [3]:
import gensim

# sparse matrix를 (doc, term) matrix가 아닌 (term, doc) matrix로 변경하기 위해 transpose
corpus = gensim.matutils.Sparse2Corpus(x.transpose())

In [4]:
for i, doc in enumerate(corpus):
    if i > 3: break
    print('#doc= %d:'%i, doc[:10], '...\n')

#doc= 0: [] ...

#doc= 1: [(4592, 1), (818, 2), (697, 1), (3202, 1), (4678, 1), (530, 3), (1188, 1), (1605, 1), (3152, 1), (2104, 1)] ...

#doc= 2: [(2552, 1), (4671, 1), (1174, 1), (3678, 1), (4615, 2), (447, 1), (4740, 1), (1871, 1), (3131, 1), (3679, 1)] ...

#doc= 3: [(3868, 1), (3513, 1), (2701, 1), (2117, 1), (756, 1), (2858, 1), (1890, 1), (2530, 1), (2189, 1), (469, 1)] ...



In [5]:
len(corpus)

26368

## Dictionary Formats

Gensim은 우리가 이용하는 Vectorizer.vocabulary_와 같은 역할로 Dictionary라는 class를 이용하고 있음. 이는 topic modeling의 결과를 0, 1, 2와 같은 term index가 아닌 단어로 보여주는 역할을 함. Format이 조금 다르기 때문에 Vectorizer를 이용하여 term frequency matrix를 만든 뒤, 그 결과를 이용해 Dictionary format의 파일을 만들어낼 것임

아래와 같은 text라는 list of list of str 형식의 docs가 있을 때, 이를 이용하여 dictionary를 만들고 저장하는 방법은 아래와 같음

    texts = [['human', 'interface', 'computer'],
    ['survey', 'user', 'computer', 'system', 'response', 'time'],
    ['system', 'user', 'interface', 'system'],
    ['user', 'response', 'time'],
    ['tree'],
    ['graph', 'trees'],
    ['graph', 'minors', 'trees'],
    ['graph', 'minors', 'survey']]
    
    dictionary = corpora.Dictionary(texts)
    dictionary.save_as_text('dictionary')
    
dictionary라는 파일에 Dictionary의 정보가 저장됨. 저장된 내용은 아래와 같음

    9
    2	computer	2
    8	eps	2
    10	graph	3
    0	human	2
    1	interface	2
    11	minors	2
    6	response	2
    3	survey	2
    5	system	3
    7	time	2
    9	trees	3
    4	user	3
    
맨 첫줄에 num docs가 들어있고, 그 다음줄부터 (term index, term, document frequency)가 tab구분이 되어 저장됨. Vectorizer.vocabulary_와 x를 이용하면 동일한 형식의 dictionary를 만들 수 있음

Sparse Matrix는 아래처럼 세 개의 list가 평행하게 움직임

    rows = [1, 1, 2, 5, ...]
    cols = [13, 734, 0, 4, ...]
    data = [1, 3, 1, 9, ...]
    
이는 아래의 의미임

    (1, 13) = 1
    (1, 734) = 3
    
data는 term frequency인데, 이 값이 1보다 크면 모두 1로 바꾸면 됨. 이 부분이 아래의 코드임

    row, col = x.nonzero()
    data = [1] * len(row)
    x_boolean = csr_matrix((data, (row, col)))
    
그 다음은 column을 중심으로 row를 합치면 단어의 document frequency가 구해짐

### 사전을 저장

In [6]:
from scipy.sparse import csr_matrix

row, col = x.nonzero()
data = [1] * len(row)
x_boolean = csr_matrix((data, (row, col)))
df = x_boolean.sum(axis = 0)

n_doc = x.shape[0]
word2index = vectorizer.vocabulary_
df = df.tolist()[0]

In [7]:
len(df)

4760

In [8]:
n_doc

26368

In [9]:
with open(dictionary_fname, 'w', encoding='utf-8') as f:
    f.write('%d\n'%n_doc)
    for word, idx in word2index.items():
        f.write('%d\t%s\t%d\n'%(idx, word, df[idx]))

In [10]:
LdaModel?

Object `LdaModel` not found.


In [11]:
from gensim.corpora import Dictionary
dictionary = Dictionary.load_from_text(dictionary_fname)

In [12]:
dictionary.doc2bow(['정부', '정책', '자유'])

[(3378, 1), (3588, 1), (3614, 1)]

데이터를 gensim LDA format으로 맞추는 일이 끝남. dictionary와 corpus를 이용하여 LDA를 학습함

num_topics는 토픽의 갯수를 정하는 부분임. id2word를 입력하지 않으면 단어가 term index로 출력됨

In [13]:
LdaModel?

Object `LdaModel` not found.


In [14]:
len(corpus)

26368

In [15]:
from gensim.models import LdaModel
import pickle

if TRAIN_LDA:
    ldamodel = LdaModel(corpus=corpus, num_topics=50, id2word=dictionary)
    
    with open(ldamodel_fname, 'wb') as f:
        pickle.dump(ldamodel, f)
        
else:
    with open(ldamodel_fname, 'rb') as f:
        ldamodel = pickle.load(f)

print_topic은 특정topic에 대하여 설명력이 좋은 (topic probability가 높은) topn개의 단어를 prob.와 함께 출력해줌

In [16]:
ldamodel.print_topic(10, topn=5)

'0.028*"웃음" + 0.023*"있습니다" + 0.019*"우리" + 0.014*"정부" + 0.013*"지원"'

get_topic_terms(topic_id)를 하면 term index가 출력되기 때문에 print_topic()의 결과에서 단어만 선택하는 sparse_topic_words()함수를 만들어 50개의 토픽에 대하여 각각 대표단어를 뽑아낼 수 있음

In [17]:
ldamodel.get_topic_terms(1)

[(2863, 0.059436067315897227),
 (355, 0.031683337759101725),
 (1005, 0.027690090448755079),
 (2775, 0.021355080690100687),
 (1117, 0.019250735874195023),
 (2430, 0.017368964752001712),
 (405, 0.015494830637440482),
 (3400, 0.014352134978649091),
 (2359, 0.011918621148862078),
 (4700, 0.011850607329534475)]

In [18]:
def parse_topic_words(topic_str):
    return [col.split('*"')[1][:-1] for col in topic_str.split(' + ')]

In [20]:
for i in range(50):
    print('#topic= %d:'%i, parse_topic_words(ldamodel.print_topic(i, topn=10)))
    if i % 5 == 4: print()

#topic= 0: ['개헌', '대통령', '지금', '국민', '문제', '경제', '정치', '상황', '생각', '논의']
#topic= 1: ['영화', '감독', '남자', '여자', '닥터', '스트레인지', '개봉', '작품', '수상', '활약']
#topic= 2: ['요리', '고양이', '코미디', '화면', '관객', '미스', '이동', '24', '대화', '영화']
#topic= 3: ['저작권자', '전남', '경남', '광주', '24', '선정', '한강', '동상', '헤럴드경제', '순천']
#topic= 4: ['뉴스1', '보험', '2014년', '배포', '회의', '당시', '할인', '결정', '기억', '확인']

#topic= 5: ['르노삼성', '검색', '24', '웹툰', '금괴', '인천공항', '출국', '인천국제공항', '사이트', '뉴스']
#topic= 6: ['기업', '회장', '사업', '삼성', '한국', '지원', '창업', '경영', '임금', '케미']
#topic= 7: ['북한', '미국', '정부', '외교', '대북', '대화', '러시아', '문제', '압박', '북핵']
#topic= 8: ['시장', '중국', '미국', '인체', '인천', '공장', '국내', '제품', '생산', '글로벌']
#topic= 9: ['한국', '작품', '세계', '사로', '예술', '작가', '문화', '인간', '도시', '자리']

#topic= 10: ['웃음', '있습니다', '우리', '정부', '지원', '경제', '확대', '국민', '예산', '합니다']
#topic= 11: ['참석', '24', '오후', '대표', '북한', '금융', '회고록', '장관', '문재인', '헤어질까']
#topic= 12: ['24', '한남동', '2016', '세번째', '경찰', '모모', '집행', '티켓', '오후', '영장']
#topic= 13: ['중앙', '중국

gensim에서 단어의 topic vector를 직접적으로 찾아주는 함수가 구현되어 있지 않음. 하지만 LdaModel[bow_model]을 넣으면 bow_model에 대한 topic vector를 출력해줌. bow_model에 단어 한 개를 넣으면 해당 단어의 topic vector를 알 수 있음

bow_model은 [(term id, weight), (term id, weight), ...] 형식임